In [3]:
import datajoint as dj
from getpass import getpass

In [6]:
def clone_conn(conn):
    conn_info = conn.conn_info
    return dj.Connection(host=conn_info['host'], user=conn_info['user'], password=conn_info['passwd'],
                             init_fun=conn.init_fun, use_tls=conn_info['ssl'])
    

In [7]:
conn2 = clone_conn(dj.conn())

Connecting eywalker@tutorial-db.datajoint.io:3306


In [9]:
schema = dj.schema('eywalker_table')

schema_clone = dj.schema('eywalker_table', connection=conn2)

In [21]:
@schema
class Source(dj.Lookup):
    definition = """
    id: int    # some id
    """
    contents = zip([0, 1, 2, 3, 4])

    
@schema
class SuperCompute(dj.Computed):
    definition = """
    -> Source
    ---
    times2: int   # twice the id...so hard...
    """
    
    def make(self, key):
        cache_key = key.copy()
        cache_key['intermediate_value'] = key['id'] * 2  # result of a tough computation
        Cache.insert1(cache_key) # this is NOT in transaction and thus immediately completes!
        
        raise Exception('so long that it timed out...')
        key['times2'] = cache_key['intermediate_value']
        self.insert1(key)
    
@schema_clone
class Cache(dj.Manual):
    definition = """
    id: int   # some id
    ---
    intermediate_value: int  # intermediate 
    """

In [22]:
Cache()

id some id,intermediate_value intermediate


In [23]:
SuperCompute.populate()

Exception: so long that it timed out...

In [24]:
Cache()

id some id,intermediate_value intermediate
0,0
